<a href="https://colab.research.google.com/github/domingues100/Water-Level-Detection---IA/blob/main/Object%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Some initializations**

Install detectron2 and clone cocosplit

-Restart environment after that

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!git clone https://github.com/akarazniewicz/cocosplit.git
!pip install funcy
!pip install scikit-multilearn

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-pkh4uvd9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-pkh4uvd9
  Resolved https://github.com/facebookresearch/detectron2.git to commit e70b9229d77aa39d85f8fa5266e6ea658e92eed3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 800.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 464.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6120356 sha256=d5da8f7321469247af32afe5bcb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TRAIN MODEL

***Libraries***

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data.datasets import register_coco_instances, load_coco_json
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from google.colab.patches import cv2_imshow
import cv2
import os
import shutil

***Define some augmentations***

In [ ]:
class CustomAugmentation(T.Augmentation):
    def get_transform(self, image):
        transforms = [
            T.RandomFlip(prob=0.5, horizontal=True),
            T.RandomBrightness(0.5, 1.5),
            T.RandomContrast(0.5, 1.5),
            T.RandomRotation(angle=[-45, 45]),
            T.RandomNoise(intensity=0.15),
            T.RandomShadow(intensity=0.5),
            T.RandomZoom(scale=(0.3, 1.8)),
            T.ElasticTransform(alpha=1.2, sigma=0.2),
            T.CropAndPad(percent=(0, 0.4))
        ]
        return T.TransformList(transforms)

***Split Coco***

In [ ]:
#name of the test
teste = "TEST NAME"

#split test and train using cocosplit  --multi-class to preserv class distribuitions
!python cocosplit/cocosplit.py -s 0.7 "PATH TO JSON" train.json test.json #--having-annotations --multi-class

#move train and test to desired folders
archives = ['train.json', 'test.json']

for archive in archives:
    os.makedirs(f"PATH TO CREATE/{teste}", exist_ok=True)
    origin_path = os.path.join("/content/", archive)
    final_path = os.path.join(f"PATH TO SAVE/{teste}", archive)

    shutil.copy(origin_path, final_path)

Saved 149 entries in train.json and 65 in test.json


In [ ]:
teste_json = f"SAME AS PATH TO SAVE/{teste}/test.json"
train_json = f"SAME AS PATH TO SAVE/{teste}/train.json"
images = "IMAGES PATH"

register_coco_instances(f"NAME_{teste}", {}, teste_json, images)
register_coco_instances(f"NAME_{teste}", {}, train_json, images)

DatasetCatalog.register("NAME", lambda: register_coco_instances(f"NAME_{teste}", {}, teste_json, images), )
DatasetCatalog.register("NAME", lambda: register_coco_instances(f"NAME_{teste}", {}, train_json, images))

teste_dataset_no_crop_metadata = MetadataCatalog.get(f"NAME_{teste}")
train_dataset_no_crop_metadata = MetadataCatalog.get(f"NAME_{teste}")

In [ ]:
train_dataset_no_crop_metadata

namespace(name='train_dataset_no_crop_teste_retina_net3',
          json_file='/content/drive/MyDrive/IC_Machine_Learning/Deteccao_detectron2/teste_retina_net3/train.json',
          image_root='/content/drive/MyDrive/IC_Machine_Learning/images',
          evaluator_type='coco')

In [ ]:
import gc
gc.collect()

11997

In [ ]:
#https://detectron2.readthedocs.io/en/latest/modules/config.html
#https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md
#https://github.com/facebookresearch/detectron2/blob/main/configs/COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml
#https://colab.research.google.com/github/Tony607/detectron2_instance_segmentation_demo/blob/master/Detectron2_custom_coco_data_segmentation.ipynb#scrollTo=7unkuuiqLdqd
#https://medium.com/@chengweizhang2012/how-to-train-detectron2-with-custom-coco-datasets-4d5170c9f389

cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) #FOR RESNET
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml")) #FOR RETINANET
cfg.DATASETS.TRAIN = (f"NAME_{teste}",)
cfg.DATASETS.TEST = (f"NAME_{teste}",)
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # initialize from model zoo for resnet
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")  # initialize from model zoo for retinanet
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 200
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.MAX_SIZE_TRAIN = 224
cfg.INPUT.MIN_SIZE_TEST = 224
cfg.INPUT.MAX_SIZE_TEST = 224

cfg.INPUT.AUGMENTATIONS = [("CustomAugmentation", CustomAugmentation())]

OUTPUT_DIR = f"/OUTPUT_PATH/{teste}"
os.makedirs(OUTPUT_DIR, exist_ok=True)
cfg.OUTPUT_DIR = OUTPUT_DIR

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[02/07 01:45:07 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

  pixel_mean
  pixel_std


[02/07 01:45:07 d2.engine.train_loop]: Starting training from iteration 0
[02/07 01:48:13 d2.utils.events]:  eta: 0:27:15  iter: 19  total_loss: 1.744  loss_cls: 1.359  loss_box_reg: 0.3678    time: 9.0659  last_time: 9.5092  data_time: 8.7087  last_data_time: 9.1361   lr: 9.5905e-05  max_mem: 14821M
[02/07 01:51:10 d2.utils.events]:  eta: 0:24:22  iter: 39  total_loss: 0.4796  loss_cls: 0.3251  loss_box_reg: 0.148    time: 8.9442  last_time: 9.2403  data_time: 8.3986  last_data_time: 8.7228   lr: 0.00019581  max_mem: 14821M
[02/07 01:54:04 d2.utils.events]:  eta: 0:21:15  iter: 59  total_loss: 0.2807  loss_cls: 0.1819  loss_box_reg: 0.1121    time: 8.8563  last_time: 7.9243  data_time: 8.2662  last_data_time: 7.5034   lr: 0.00029571  max_mem: 14821M
[02/07 01:57:01 d2.utils.events]:  eta: 0:18:15  iter: 79  total_loss: 0.2243  loss_cls: 0.1302  loss_box_reg: 0.08479    time: 8.8548  last_time: 7.7764  data_time: 8.4385  last_data_time: 7.3466   lr: 0.00039561  max_mem: 14821M
[02/07 0

***Evaluate the perfomance***

In [ ]:
evaluator = COCOEvaluator(f"NAME_{teste}", cfg, False, output_dir=f"PATH_OUTPUT/{teste}")
val_loader = build_detection_test_loader(cfg, f"NAME_{teste}")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

WARNING [02/07 02:23:02 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[02/07 02:23:02 d2.data.datasets.coco]: Loaded 65 images in COCO format from /content/drive/MyDrive/IC_Machine_Learning/Deteccao_detectron2/teste_retina_net3/test.json
[02/07 02:23:02 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(224, 224), max_size=224, sample_style='choice')]
[02/07 02:23:02 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[02/07 02:23:02 d2.data.common]: Serializing 65 elements to byte tensors and concatenating them all ...
[02/07 02:23:02 d2.data.common]: Serialized dataset takes 0.02 MiB
[02/07 02:23:02 d2.evaluation.evaluator]: Start inference on 65 batches
[02/07 02:23:09 d2.evaluation.evaluator]: Inference done 11/65. Dataloading: 0.5974 s/iter. Inference: 0.0776 s/iter. Eval:

Para visualização das predições

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, f"PATH/{teste}/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
predictor = DefaultPredictor(cfg)
best_predictions = []
names = []
teste_dataset_dicts = load_coco_json(teste_json, images)

output_images_dir = f"OUTPUT_IMAG_DIR/{teste}/"
os.makedirs(output_images_dir, exist_ok=True)

for d in teste_dataset_dicts:
    im = cv2.imread(d["file_name"])
    names.append(d["file_name"])
    outputs = predictor(im)

    detections = outputs["instances"].to("cpu")

    if len(detections) > 0:
        scores = detections.scores.tolist()
        max_score_idx = scores.index(max(scores))
        best_detection = detections[max_score_idx]

        # Save the best detection for the current image
        best_predictions.append(best_detection.pred_boxes.tensor.cpu().numpy())

        v = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get(f'NAME{teste}'), scale=0.3, instance_mode=1)
        out = v.draw_instance_predictions(best_detection)

        # Save the image with the bounding boxes
        output_image_path = os.path.join(output_images_dir, os.path.basename(d["file_name"]))
        cv2.imwrite(output_image_path, out.get_image()[:, :, ::-1])

        cv2_imshow(out.get_image()[:, :, ::-1])

print(f"Images saved in: {output_images_dir}")

In [ ]:
import pandas as pd
df = pd.DataFrame({"IMAGENS": names, "BBOX": best_predictions})
df.to_csv(f"OUT_DIR/{teste}/predicoes.csv", index = False)

# Load Model and test


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer


cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "SAVE_PATH/model_final.pth"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 200
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.MAX_SIZE_TRAIN = 224
cfg.INPUT.MIN_SIZE_TEST = 224
cfg.INPUT.MAX_SIZE_TEST = 224
cfg.INPUT.AUGMENTATIONS = [("CustomAugmentation", CustomAugmentation())]

model = DefaultPredictor(cfg).model
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)


{'trainer': {'iteration': 199,
  'hooks': {'LRScheduler': {'base_lrs': [0.001], 'last_epoch': 200}},
  '_trainer': {'iteration': 199,
   'optimizer': {'state': {0: {'momentum_buffer': tensor([[[[ 5.4489e-04]],
      
               [[ 7.8342e-06]],
      
               [[ 8.2637e-04]],
      
               ...,
      
               [[ 2.7083e-04]],
      
               [[ 1.0643e-03]],
      
               [[ 6.7073e-04]]],
      
      
              [[[-1.2090e-03]],
      
               [[-1.1588e-03]],
      
               [[ 1.3129e-04]],
      
               ...,
      
               [[-1.4667e-03]],
      
               [[ 5.5796e-05]],
      
               [[-8.3593e-04]]],
      
      
              [[[-6.4378e-04]],
      
               [[-1.2689e-03]],
      
               [[-1.2853e-03]],
      
               ...,
      
               [[-1.9477e-03]],
      
               [[ 1.5475e-04]],
      
               [[ 6.8147e-04]]],
      
      
              .

In [ ]:
#as imagens de treino originais para testar
teste_json = f"PATH TO /test.json"
images_originais = "PATH TO IMAGES"

In [ ]:
register_coco_instances("NAME", {}, teste_json, images_originais)

DatasetCatalog.register("NAME", lambda: register_coco_instances("NAME", {}, teste_json, images_originais), )

teste_dataset_no_crop_metadata = MetadataCatalog.get("NAME")

***Test name***

In [ ]:
teste = "TEST NAME"


***Evaluator***

In [ ]:
evaluator = COCOEvaluator("NAME", cfg, False, output_dir=f"PATH TO  OUT/{teste}")
val_loader = build_detection_test_loader(cfg, "NAME")
print(inference_on_dataset(model, val_loader, evaluator))

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.893
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.893
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.911
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.911
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.911
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.5

***only Make predictions***

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "path to model/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
predictor = DefaultPredictor(cfg)
best_predictions = []
names = []
teste_dataset_dicts = load_coco_json(teste_json, images_originais)

output_images_dir = f"OUPUT DIR/{teste}"
os.makedirs(output_images_dir, exist_ok=True)

for d in teste_dataset_dicts:
    im = cv2.imread(d["file_name"])
    names.append(d["file_name"])
    outputs = predictor(im)

    detections = outputs["instances"].to("cpu")

    if len(detections) > 0:
        scores = detections.scores.tolist()
        max_score_idx = scores.index(max(scores))
        best_detection = detections[max_score_idx]

        # Save the best detection for the current image
        best_predictions.append(best_detection.pred_boxes.tensor.cpu().numpy())

        v = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get(f'NAME'), scale=0.3, instance_mode=1)
        out = v.draw_instance_predictions(best_detection)

        # Save the image with the bounding boxes
        output_image_path = os.path.join(output_images_dir, os.path.basename(d["file_name"]))
        cv2.imwrite(output_image_path, out.get_image()[:, :, ::-1])

        cv2_imshow(out.get_image()[:, :, ::-1])

print(f"Images saved in: {output_images_dir}")

***Make preds and crop de object***

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
predictor = DefaultPredictor(cfg)
best_predictions = []
gt_bbox = []
names = []
teste_dataset_dicts = load_coco_json(teste_json, images_originais)

output_images_dir = f"OUTPUT DIR/{teste}"
os.makedirs(output_images_dir, exist_ok=True)

for d in teste_dataset_dicts:
    im = cv2.imread(d["file_name"])

    #saving gt bbox
    gt_bbox.append(d["annotations"][0]["bbox"])
    names.append(d["file_name"])
    outputs = predictor(im)

    detections = outputs["instances"].to("cpu")

    if len(detections) > 0:
        scores = detections.scores.tolist()
        max_score_idx = scores.index(max(scores))
        best_detection = detections[max_score_idx]
        bbox = best_detection.pred_boxes.tensor.cpu().numpy()[0]

        #saving best_prediction bbox
        best_predictions.append(bbox)
        x, y, w, h = bbox
        x, y, w, h = int(x), int(y), int(w), int(h)

        cropped_image = im[y:h, x:w]

        # Salvar o recorte
        output_image_path = os.path.join(output_images_dir, os.path.basename(d["file_name"]))
        cv2.imwrite(output_image_path, cropped_image)

        # Visualizar a imagem recortada
        cv2_imshow(cropped_image)

print(best_predictions)
print(gt_bbox)

***Code for IoU calculation***

In [ ]:
def gt_bbox_coco_format(gt_bbox):
  gt_bbox_copy = gt_bbox.copy()
  for item in gt_bbox_copy:
    x, y, w, h = item
    item[2] = x+w
    item[3] = y+h
  return gt_bbox_copy

def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Parameters:
    - box1: Tuple (x1, y1, x2, y2) representing the coordinates of the first bounding box.
    - box2: Tuple (x1, y1, x2, y2) representing the coordinates of the second bounding box.

    Returns:
    - IoU: Intersection over Union value.
    """

    # Calculate the intersection coordinates
    x1_i = max(box1[0], box2[0])
    y1_i = max(box1[1], box2[1])
    x2_i = min(box1[2], box2[2])
    y2_i = min(box1[3], box2[3])

    # Calculate the intersection area
    intersection_area = max(0, x2_i - x1_i + 1) * max(0, y2_i - y1_i + 1)

    # Calculate the union area
    area_box1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area_box2 = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = area_box1 + area_box2 - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    # append iou in a list
    iou_list.append(iou)


#list to store iou
iou_list = []

formated_gt_bbox = gt_bbox_coco_format(gt_bbox)

for item1, item2 in zip(formated_gt_bbox, best_predictions):
  calculate_iou(item1, item2)

mean_iou = sum(iou_list)/len(iou_list)
print(mean_iou)


0.929778464676807
